## Topics

- Runnable
- Prompt Template
- Sequential memory
- Output parsing

In [ ]:
%pip install langchain langchain-openai

In [ ]:
import os
from langchain_openai import ChatOpenAI  # pip install langchain-openai

llm = ChatOpenAI(
    openai_api_key="", 
    temperature=.75, 
    max_tokens=1024, 
    request_timeout=30
)

In [ ]:
llm.invoke("test")

In [ ]:
from langchain.prompts import ChatPromptTemplate  # pip install langchain

prompt = ChatPromptTemplate.from_messages([
    ("system", "Act as a world class Machine Learning engineer. Use english language. End your answers with a reference to the beauty of using data science in any decision you make."),
    ("user", "{input}")
])

# concatenazione del prompt al modello
chain = prompt | llm

## Runnable Interface

To simplify the creation of even very complex event/execution chains, all LangChain components implement a "runnable" protocol through a common interface that allows any component to be used in a standard way. Below are the three main methods:

* **stream** - send partial responses as they are generated
* **invoke** - execute the chain on a single input
* **batch** - execute the chain on multiple inputs

### Input and Output of Main Components
<img src="assets/componenti_io.png" width="600">

One of the advantages of Runnable interfaces is that runnable components can be chained together in execution sequences, allowing the output of one component to automatically become the input to another. The *pipe* command **|** is used for this purpose in LCEL (LangChain Expression Language), enabling the creation of runnable components from other runnable components by configuring them into a sequence that will work synergistically.


In [ ]:
chain.invoke({"input": "hello!"})

# ConversationBufferMemory

[`ConversationBufferMemory`](https://api.python.langchain.com/en/latest/memory/langchain.memory.buffer.ConversationBufferMemory.html) is a tool in LangChain that helps keep track of a conversation. It stores the messages exchanged between the user and the AI so that the AI can remember what has been said earlier. This helps the AI maintain context and continuity in the conversation.

`ConversationBufferMemory` is a type of sequential memory in Langchain:

<img src="assets/sequential-memory.png" width="300" />


Here’s a basic example of how to add messages to a `ConversationBufferMemory` and how to get back the messages:

In [ ]:
from langchain.memory import ConversationBufferMemory

# Create a new conversation memory
memory = ConversationBufferMemory()

# Add user and AI messages to the memory
memory.chat_memory.add_user_message("Hello")
memory.chat_memory.add_ai_message("Hi! How you doin'?")
memory.chat_memory.add_user_message("Fine, thanks.")

print(memory.load_memory_variables({})['history'])

In [ ]:
memory = ConversationBufferMemory(return_messages=True)

# Add user and AI messages to the memory
memory.chat_memory.add_user_message("Hello")
memory.chat_memory.add_ai_message("Hi! How you doin'?")
memory.chat_memory.add_user_message("Fine, thanks.")

memory.load_memory_variables({})

# Introduction to PromptTemplate

The `PromptTemplate` is a powerful feature designed to streamline and standardize the creation of prompts for various applications, such as chatbots, automated responses, or data entry forms. It provides a structured format that can be reused across different scenarios, ensuring consistency and efficiency in how inputs are solicited and processed.



In [ ]:
# dynamic template and use of a Memory Buffer

template = """Act as a data scientist answering to every question with references to the beauty of Data Science.
Conversation:
{chat}

New question: {question}
Answer:"""

from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(template)

memory = ConversationBufferMemory(memory_key="chat")

from langchain.chains import LLMChain

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [ ]:
conversation.invoke({"question": "Hello, i lake the orange color."})

In [ ]:
print(memory.load_memory_variables({})['chat'])

In [ ]:
conversation.invoke({"question": "Tell me 3 fruits of my favourite color"})

In [ ]:
print(memory.load_memory_variables({})['chat'])

## LLM output parsing

<a href="https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/quick_start/" target="_blank">source</a>

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.

**Output parsers** are classes that help *structure language model responses*. 

There are two main methods an output parser must implement:

- "Get format instructions": A method which returns a string containing instructions for how the output of a language model should be formatted.
- "Parse": A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

And then one optional one:

- "Parse with prompt": A method which takes in a string (assumed to be the response from a language model) and a prompt (assumed to be the prompt that generated such a response) and parses it into some structure. The prompt is largely provided in the event the OutputParser wants to retry or fix the output in some way, and needs information from the prompt to do so.

In [ ]:
# Create a full chain with a prompt, a model and an output parser

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field, validator

class User(BaseModel):
    id: int = Field(description="user identification number")
    name: str = Field(description="user name")
    mail: str = Field(description="user mail address")
    

# create a chain with a Pydantic object parser
# create a chain with a JsonOutputParser